# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.1.0.4/9764/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

['/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-01.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-02.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-03.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-04.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-05.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-06.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-07.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-08.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-09.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-10.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-11.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-12.csv',
 '/home/runner/work/dask-exa

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1040,Jerry,-0.09283830034372209,-0.6137694291975466
2000-01-01 00:00:01,1029,Bob,-0.9301215521403607,0.895213226045229
2000-01-01 00:00:02,941,Bob,0.23489049783038163,0.7165773843157339
2000-01-01 00:00:03,999,Patricia,-0.28220386774692763,0.06523652758797471
2000-01-01 00:00:04,985,Frank,-0.8777466864040762,-0.10685736632813558
2000-01-01 00:00:05,1006,Xavier,0.6718911996426065,-0.05910630091711666
2000-01-01 00:00:06,1012,Yvonne,-0.35876141939845985,-0.7092309664603245
2000-01-01 00:00:07,1026,Frank,-0.47461217038032855,-0.218033477187082
2000-01-01 00:00:08,1024,Hannah,0.7680752623314022,-0.16930162117439762


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,973,Oliver,0.5127211956411899,0.6500816491598798
2000-01-30 00:00:01,1015,Oliver,-0.24180799227773164,0.9760048280207327
2000-01-30 00:00:02,1010,Kevin,-0.3314358923441476,0.08393260184178564
2000-01-30 00:00:03,1015,Wendy,0.6419201827179413,-0.7976121218318282
2000-01-30 00:00:04,1049,Victor,0.8123696270022782,0.14052704447114572
2000-01-30 00:00:05,983,Wendy,-0.08814685187335036,0.9066161835095308
2000-01-30 00:00:06,1040,Hannah,0.758293115531208,0.3667933274123347
2000-01-30 00:00:07,984,Dan,-0.20296394702933052,0.32218445226511583
2000-01-30 00:00:08,979,Norbert,0.940966270596012,0.11121509653232042


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1040,Jerry,-0.092838,-0.613769
1,2000-01-01 00:00:01,1029,Bob,-0.930122,0.895213
2,2000-01-01 00:00:02,941,Bob,0.234890,0.716577
3,2000-01-01 00:00:03,999,Patricia,-0.282204,0.065237
4,2000-01-01 00:00:04,985,Frank,-0.877747,-0.106857


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1040,Jerry,-0.092838,-0.613769
1,2000-01-01 00:00:01,1029,Bob,-0.930122,0.895213
2,2000-01-01 00:00:02,941,Bob,0.234890,0.716577
3,2000-01-01 00:00:03,999,Patricia,-0.282204,0.065237
4,2000-01-01 00:00:04,985,Frank,-0.877747,-0.106857


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.62 s, sys: 519 ms, total: 4.14 s
Wall time: 2.89 s


name
Alice       0.001516
Bob         0.003195
Charlie     0.000623
Dan        -0.000731
Edith      -0.000351
Frank       0.002086
George      0.001113
Hannah     -0.001602
Ingrid      0.001184
Jerry       0.000764
Kevin      -0.001170
Laura       0.000018
Michael     0.000394
Norbert     0.003474
Oliver     -0.001964
Patricia   -0.001930
Quinn       0.002254
Ray         0.003143
Sarah      -0.002672
Tim         0.000475
Ursula     -0.001093
Victor     -0.000150
Wendy      -0.002209
Xavier     -0.000869
Yvonne     -0.003750
Zelda      -0.003297
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.08 s, sys: 151 ms, total: 1.23 s
Wall time: 912 ms


name
Alice       0.001516
Bob         0.003195
Charlie     0.000623
Dan        -0.000731
Edith      -0.000351
Frank       0.002086
George      0.001113
Hannah     -0.001602
Ingrid      0.001184
Jerry       0.000764
Kevin      -0.001170
Laura       0.000018
Michael     0.000394
Norbert     0.003474
Oliver     -0.001964
Patricia   -0.001930
Quinn       0.002254
Ray         0.003143
Sarah      -0.002672
Tim         0.000475
Ursula     -0.001093
Victor     -0.000150
Wendy      -0.002209
Xavier     -0.000869
Yvonne     -0.003750
Zelda      -0.003297
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.09 s, sys: 58.5 ms, total: 1.14 s
Wall time: 883 ms


name
Alice       0.001516
Bob         0.003195
Charlie     0.000623
Dan        -0.000731
Edith      -0.000351
Frank       0.002086
George      0.001113
Hannah     -0.001602
Ingrid      0.001184
Jerry       0.000764
Kevin      -0.001170
Laura       0.000018
Michael     0.000394
Norbert     0.003474
Oliver     -0.001964
Patricia   -0.001930
Quinn       0.002254
Ray         0.003143
Sarah      -0.002672
Tim         0.000475
Ursula     -0.001093
Victor     -0.000150
Wendy      -0.002209
Xavier     -0.000869
Yvonne     -0.003750
Zelda      -0.003297
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html